In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [99]:
user_df=pd.read_csv('takehome_users.csv')
engagement_df=pd.read_csv('takehome_user_engagement.csv')

In [100]:
user_df.head()


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [101]:
user_df.describe()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
mean,6000.50000,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,1.363195e+09,0.000000,0.000000,29.000000,3058.000000
50%,6000.50000,1.382888e+09,0.000000,0.000000,108.000000,5954.000000
75%,9000.25000,1.398443e+09,0.000000,0.000000,238.250000,8817.000000
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


In [102]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [103]:
engagement_df.head()


,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [104]:
engagement_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [105]:
engagement_df.describe()

,user_id,visited
count,207917.000000,207917.0
mean,5913.314197,1.0
std,3394.941674,0.0
min,1.000000,1.0
25%,3087.000000,1.0
50%,5682.000000,1.0
75%,8944.000000,1.0
max,12000.000000,1.0


In [106]:
len(engagement_df.user_id.unique())

8823

In [107]:
engagement_df['time_stamp']=engagement_df.time_stamp.apply(lambda x: 
                            datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
engagement_df=engagement_df.set_index('time_stamp')

dfday=engagement_df.groupby('user_id').resample('1D').sum()

In [108]:
dfday=dfday[dfday.user_id>0]
dfday[dfday.visited>1]


,,user_id,visited
user_id,time_stamp,,


Not a single user visited more than once in a day

In [109]:
dfweek=engagement_df.groupby('user_id').resample('7D').sum()

In [110]:
dfweek=dfweek[dfweek.user_id>0]
dfweek.user_id=dfweek.user_id/dfweek.visited


In [111]:
engaged=[]
dic={}
tvisits={}
for i, v in zip(dfweek.user_id,dfweek.visited):
    
    if v>2:
        dic[i]=1
        engaged.append(i)
    if i not in engaged:
        dic[i]=0
        
    

In [118]:
engaged=pd.DataFrame(data= dic.values(),index=dic.keys(), columns=['engaged'])
engaged=engaged.merge(engagement_df.groupby('user_id').sum(), left_on=engaged.index, right_index=True)
engaged=engaged[['engaged','visited']]


In [123]:
complete=user_df.merge(engaged, left_on=user_df.object_id, right_on=engaged.index)
complete=complete.loc[:,'object_id':'visited']
complete['count']=1
#complete['visited']=engaged.loc[:,'visited']
complete.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,engaged,visited,count
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0,1,1
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1,14,1
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0,1,1
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0,1,1
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0,1,1


In [127]:
complete[['opted_in_to_mailing_list','enabled_for_marketing_drip','engaged', 'visited']].groupby('engaged').mean().sort_values('engaged', ascending=False)

,opted_in_to_mailing_list,enabled_for_marketing_drip,visited
engaged,,,
1,0.261549,0.154212,133.521060
0,0.250442,0.151544,1.547272


In [125]:
x=complete[['engaged','org_id','visited']].groupby('org_id').mean().sort_values('engaged', ascending=False)
y=complete[['count','org_id']].groupby('org_id').sum()
z=x.merge(y,left_on=x.index, right_on=y.index, right_index=True)
z

,key_0,engaged,visited,count
org_id,,,,
235,235,0.600000,144.500000,10
273,273,0.555556,92.555556,9
387,387,0.545455,66.272727,11
289,289,0.545455,92.545455,11
306,306,0.500000,50.583333,12
399,399,0.500000,95.100000,10
400,400,0.500000,161.833333,6
270,270,0.461538,56.769231,13
280,280,0.454545,49.272727,11


In [126]:
x=complete[['engaged','creation_source','visited']].groupby('creation_source').mean().sort_values('engaged', ascending=False)
y=complete[['count','creation_source']].groupby('creation_source').sum()
z=x.merge(y,left_on=x.index, right_on=y.index, right_index=True)
z

,key_0,engaged,visited,count
creation_source,,,,
GUEST_INVITE,GUEST_INVITE,0.209068,29.468514,1588
PERSONAL_PROJECTS,PERSONAL_PROJECTS,0.188482,25.159686,764
ORG_INVITE,ORG_INVITE,0.158093,21.985885,3188
SIGNUP_GOOGLE_AUTH,SIGNUP_GOOGLE_AUTH,0.156679,24.738628,1385
SIGNUP,SIGNUP,0.144889,19.781349,1898


In [132]:
complete[['engaged','visited']].groupby(complete.visited//5*5).mean().sort_index()

,engaged,visited
visited,,
0,0.003430,1.195084
5,0.225543,6.586957
10,0.547297,11.966216
15,0.777778,16.950617
20,0.931034,22.011494
25,0.969697,26.621212
30,1.000000,31.941176
35,1.000000,36.333333
40,1.000000,42.122449
